In [55]:
from __future__ import print_function
import os, sys, gzip
import pandas as pd
import cPickle as pickle
sys.setrecursionlimit(10000)

np.random.seed(1004)   # for reproducibility

# stochastic neighbor embedding (to install --> pip install tsne)
from tsne import bh_sne

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
%matplotlib inline
from scipy.misc import imresize

from lasagne import layers, nonlinearities, updates, objectives, init 
from lasagne.layers import get_output, get_output_shape, get_all_params
import theano.tensor as T
import theano

Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, CuDNN 4007)


In [45]:
datapath='Network_1_data/Network_1_Configuration_1_knockouts.tsv'
df = pd.read_csv(datapath, delimiter='\t')
df.head()

G1        G2        G3        G4        G5        G6        G7  \
0  0.000000  0.063687  0.077386  0.063011  0.062566  0.060599  0.045470   
1  0.065022  0.000000  0.080552  0.067826  0.063314  0.063417  0.048150   
2  0.063673  0.066243  0.000000  0.059311  0.069221  0.062710  0.051341   
3  0.063687  0.062515  0.074903  0.000000  0.065504  0.067427  0.052367   
4  0.065150  0.067225  0.086093  0.068404  0.000000  0.065454  0.045319   

         G8        G9       G10      ...           G4992     G4993     G4994  \
0  0.889052  0.063623  0.047595      ...        0.005927  0.080815  0.020563   
1  0.857127  0.064651  0.045210      ...        0.007253  0.076590  0.021910   
2  0.784903  0.065072  0.041794      ...        0.005937  0.087488  0.020521   
3  0.758373  0.060590  0.040716      ...        0.007040  0.079408  0.022674   
4  0.816946  0.067466  0.045548      ...        0.007033  0.079868  0.020495   

      G4995     G4996     G4997     G4998     G4999     G5000  Unnamed: 5000  
0  0.045038  0.039849  0.042589  0.021239  0.043806  0.062454            NaN  
1  0.042727  0.043686  0.045363  0.021322  0.041736  0.063389            NaN  
2  0.044604  0.041984  0.043140  0.019675  0.044145  0.069649            NaN  
3  0.043219  0.042946  0.044174  0.021827  0.040556  0.068060            NaN  
4  0.043419  0.042411  0.041535  0.020285  0.044227  0.065812            NaN  

[5 rows x 5001 columns]

In [49]:
df.iloc[1]

G1               0.065022
G2               0.000000
G3               0.080552
G4               0.067826
G5               0.063314
G6               0.063417
G7               0.048150
G8               0.857127
G9               0.064651
G10              0.045210
G11              0.085755
G12              0.009377
G13              0.013085
G14              0.036747
G15              0.066073
G16              0.044039
G17              0.155478
G18              0.067586
G19              0.067938
G20              0.122251
G21              0.015457
G22              0.022515
G23              0.063357
G24              0.065511
G25              0.060288
G26              0.066520
G27              0.031188
G28              0.090031
G29              0.069395
G30              0.066182
                   ...   
G4972            0.043693
G4973            0.044081
G4974            0.014837
G4975            0.042556
G4976            0.059585
G4977            0.000023
G4978            0.113372
G4979       

In [51]:
num_genes = df.shape[0]

ge = np.zeros((num_genes, num_genes))
for i in range(num_genes):
    ge[i,:] = df.iloc[i][0:num_genes]


In [56]:
ge.dtype

dtype('float64')

In [ ]:
# perform t-SNE embedding
coord = bh_sne(ge)

In [ ]:
vis_x = coord[:,0]
vis_y = coord[:,1]
plt.figure(figsize = (10,10))
fig = plt.gcf()
fig.set_size_inches(30,30, forward=True)
plt.scatter(vis_x, vis_y, edgecolor = 'none')
plt.axis('off')

In [57]:
input_var = T.dmatrix('input')
target_var = T.dmatrix('target')

l_in = layers.InputLayer(shape=(None,5000), input_var=input_var, name='input')

l_dense1 = layers.DenseLayer(l_in, num_units=1000, W=init.GlorotUniform(), b=None, nonlinearity=None)
l_bias1 = layers.BiasLayer(l_dense1, b=init.Constant(0.05))
l_nonlin1 = layers.NonlinearityLayer(l_bias1, nonlinearity=nonlinearities.rectify)

l_dense2 = layers.DenseLayer(l_nonlin1, num_units=200, W=init.GlorotUniform(), b=None, nonlinearity=None)
l_bias2 = layers.BiasLayer(l_dense2, b=init.Constant(0.05))
l_nonlin2 = layers.NonlinearityLayer(l_bias2, nonlinearity=nonlinearities.rectify)

l_enc = layers.NonlinearityLayer(l_nonlin2, nonlinearity=None)

l_dec3 = layers.InverseLayer(l_enc, l_dense2)
l_bias3 = layers.BiasLayer(l_dec3, init.Constant(0.05))
l_nonlin3 = layers.NonlinearityLayer(l_bias3, nonlinearity=nonlinearities.rectify)

l_dec4 = layers.InverseLayer(l_nonlin3, l_dense1)
l_bias4 = layers.BiasLayer(l_dec4, init.Constant(0.05))
l_out = layers.NonlinearityLayer(l_bias4, nonlinearity=nonlinearities.rectify)


# train and validation loss
prediction = get_output(l_out)
train_loss = objectives.squared_error(prediction, input_var)
train_loss = train_loss.mean()

valid_prediction = get_output(l_out, deterministic=True)
valid_loss = objectives.squared_error(valid_prediction, input_var)
valid_loss = valid_loss.mean()

# optimizer
params = get_all_params(l_out, trainable=True)
update_op = updates.adam(train_loss, params, learning_rate=.0001)

# theano function
train_function = theano.function([input_var], train_loss, updates=update_op)
valid_function = theano.function([input_var], valid_loss)

In [ ]:
def batch_generator(X, batch_size=128, shuffle=True):
    """python generator to get a randomized minibatch"""
    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
    for start_idx in range(0, len(X)-batch_size+1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx+batch_size]
        else:
            excerpt = slice(start_idx, start_idx+batch_size)
        yield X[excerpt].astype('float32')
        
batch_size = 128
num_train_batches = len(ge) // batch_size

# train network (for short time -- MNIST is quick to train)
n_epochs = 50
for e in range(n_epochs):
    print('epoch %d' % (e+1))
    train_batches = batch_generator(ge, batch_size)

    ave_loss = 0
    for index in range(num_train_batches):
        X_batch = next(train_batches)
        train_loss = train_function(X_batch)
        ave_loss += train_loss
    print("train: %f" % float(ave_loss/num_train_batches))


epoch 1
train: 0.000785
epoch 2
train: 0.000145
epoch 3
train: 0.000078
epoch 4
train: 0.000072
epoch 5
train: 0.000072
epoch 6
train: 0.000072
epoch 7
train: 0.000072
epoch 8
train: 0.000072
epoch 9
train: 0.000072
epoch 10
train: 0.000072
epoch 11
train: 0.000072
epoch 12
train: 0.000072
epoch 13
train: 0.000072
epoch 14
train: 0.000072
epoch 15
train: 0.000072
epoch 16
train: 0.000072
epoch 17
train: 0.000072
epoch 18
train: 0.000072
epoch 19
train: 0.000072
epoch 20
train: 0.000072
epoch 21
train: 0.000072
epoch 22
